# Building a bad point pressure lift Controller for a district heating network

## BadPointPressureLiftController: Pressure Control at the Worst Point

The `BadPointPressureLiftController` is a custom controller designed for district heating networks modeled with pandapipes. Its main purpose is to maintain a minimum pressure difference at the network's "worst point"—the heat exchanger with the lowest pressure difference (Schlechtpunktregelung).

### Key Features

- **Automatic Worst Point Detection:** Identifies the heat exchanger with the lowest pressure difference where heat flow is present.
- **Pressure Regulation:** Adjusts the circulation pump's lift and flow pressures to ensure the pressure difference at the worst point meets a specified minimum target.
- **Proportional Control:** Uses a proportional gain to determine the adjustment magnitude based on the deviation from the target pressure difference.
- **Standby Mode:** If no heat flow is detected, the controller switches the pump to a standby mode with minimum lift and flow pressures.
- **Convergence Check:** Determines if the pressure difference is within a specified tolerance of the target, signaling convergence.

### Usage

- **Initialization:** The controller is initialized with the network, pump index, target pressure difference, tolerance, proportional gain, and minimum pressure settings.
- **Integration:** It can be integrated into a simulation loop, automatically adjusting pump pressures at each time step to maintain optimal network operation.

This controller is particularly useful for ensuring reliable and efficient operation in district heating systems, where maintaining a minimum pressure difference at the most critical point is essential for system stability and performance.

In [ ]:
from pandapower.control.basic_controller import BasicCtrl


class BadPointPressureLiftController(BasicCtrl):
    """
    A controller for maintaining the pressure difference at the worst point (German: Differenzdruckregelung im Schlechtpunkt) in the network.
    
    Args:
        net (pandapipesNet): The pandapipes network.
        circ_pump_pressure_idx (int, optional): Index of the circulation pump. Defaults to 0.
        target_dp_min_bar (float, optional): Target minimum pressure difference in bar. Defaults to 1.
        tolerance (float, optional): Tolerance for pressure difference. Defaults to 0.2.
        proportional_gain (float, optional): Proportional gain for the controller. Defaults to 0.2.
        min_plift (float, optional): Minimum lift pressure in bar. Defaults to 1.5.
        min_pflow (float, optional): Minimum flow pressure in bar. Defaults to 3.5.
        **kwargs: Additional keyword arguments.
    """
    def __init__(self, net, circ_pump_pressure_idx=0, target_dp_min_bar=1, tolerance=0.2, proportional_gain=0.2, min_plift=1.5, min_pflow=3.5, **kwargs):
        super(BadPointPressureLiftController, self).__init__(net, **kwargs)
        self.circ_pump_pressure_idx = circ_pump_pressure_idx
        self.target_dp_min_bar = target_dp_min_bar
        self.tolerance = tolerance
        self.proportional_gain = proportional_gain

        self.min_plift = min_plift  # Minimum pressure in bar
        self.min_pflow = min_pflow  # Minimum lift pressure in bar

        self.iteration = 0  # Add iteration counter

        self.dp_min, self.heat_consumer_idx = self.calculate_worst_point(net)

    def calculate_worst_point(self, net):
        """Calculate the worst point in the heating network, defined as the heat exchanger with the lowest pressure difference.

        Args:
            net (pandapipesNet): The pandapipes network.

        Returns:
            tuple: The minimum pressure difference and the index of the worst point.
        """
        
        dp = []

        for idx, qext, p_from, p_to in zip(net.heat_consumer.index, net.heat_consumer["qext_w"], net.res_heat_consumer["p_from_bar"], net.res_heat_consumer["p_to_bar"]):
            if qext != 0:
                dp_diff = p_from - p_to
                dp.append((dp_diff, idx))

        if not dp:
            return 0, -1

        # Find the minimum delta p where the heat flow is not zero
        dp_min, idx_min = min(dp, key=lambda x: x[0])

        return dp_min, idx_min

    def time_step(self, net, time_step):
        """Reset the iteration counter at the start of each time step.

        Args:
            net (pandapipesNet): The pandapipes network.
            time_step (int): The current time step.

        Returns:
            int: The current time step.
        """
        self.iteration = 0  # reset iteration counter
        self.dp_min, self.heat_consumer_idx = self.calculate_worst_point(net)

        return time_step

    def is_converged(self, net):
        """Check if the controller has converged.

        Args:
            net (pandapipesNet): The pandapipes network.

        Returns:
            bool: True if converged, False otherwise.
        """

        if all(net.heat_consumer["qext_w"] == 0):
            return True
        
        current_dp_bar = net.res_heat_consumer["p_from_bar"].at[self.heat_consumer_idx] - net.res_heat_consumer["p_to_bar"].at[self.heat_consumer_idx]

        # Check if the pressure difference is within tolerance
        dp_within_tolerance = abs(current_dp_bar - self.target_dp_min_bar) < self.tolerance

        if dp_within_tolerance == True:
            return dp_within_tolerance

    def control_step(self, net):
        """Adjust the pump pressure to maintain the target pressure difference.

        Args:
            net (pandapipesNet): The pandapipes network.
        """
        # Increment iteration counter
        self.iteration += 1

        """Adjust the pump pressure or switch to standby mode when heat flow is zero."""
        if all(net.heat_consumer["qext_w"] == 0):
            # Switch to standby mode
            print("No heat flow detected. Switching to standby mode.")
            net.circ_pump_pressure["plift_bar"].iloc[:] = self.min_plift  # Minimum lift pressure
            net.circ_pump_pressure["p_flow_bar"].iloc[:] = self.min_pflow  # Minimum flow pressure
            return super(BadPointPressureLiftController, self).control_step(net)

        # Check whether the heat flow in the heat exchanger is zero
        current_dp_bar = net.res_heat_consumer["p_from_bar"].at[self.heat_consumer_idx] - net.res_heat_consumer["p_to_bar"].at[self.heat_consumer_idx]
        current_plift_bar = net.circ_pump_pressure["plift_bar"].at[self.circ_pump_pressure_idx]
        current_pflow_bar = net.circ_pump_pressure["p_flow_bar"].at[self.circ_pump_pressure_idx]

        dp_error = self.target_dp_min_bar - current_dp_bar
        
        plift_adjustment = dp_error * self.proportional_gain
        pflow_adjustment = dp_error * self.proportional_gain        

        new_plift = current_plift_bar + plift_adjustment
        new_pflow = current_pflow_bar + pflow_adjustment
        
        net.circ_pump_pressure["plift_bar"].at[self.circ_pump_pressure_idx] = new_plift
        net.circ_pump_pressure["p_flow_bar"].at[self.circ_pump_pressure_idx] = new_pflow

        return super(BadPointPressureLiftController, self).control_step(net)


## Example Usage of the BadPointPressureLiftController

To demonstrate the usage of the `BadPointPressureLiftController`, we provide an example with a simple test network. The network is initialized using a `initialize_test_net` function, which sets up a district heating system with two heat consumers, a circulation pump, and several pipes and junctions.

The controller is instantiated and added to the network as follows:

```python
net = initialize_test_net()

dp_controller = BadPointPressureLiftController(net)
net.controller.loc[len(net.controller)] = [dp_controller, True, -1, -1, False, False]
```

This function performs the following steps:
- Creates a pandapipes network with water as the working fluid.
- Adds junctions for the pump, pipes, and heat exchangers.
- Installs a circulation pump with specified flow and lift pressures.
- Adds two heat consumers with configurable heat extraction and return temperatures.
- Connects all components with pipes.
- Runs an initial pipeflow calculation.
- Instantiates the `BadPointPressureLiftController` and registers it in the network's controller table.

Once the network is initialized, the controller will automatically regulate the pump pressures during simulation to maintain the minimum pressure difference at the worst point (the heat exchanger with the lowest pressure difference). This ensures reliable operation and helps prevent under-supply at critical points in the network.

In my implementation, a dp_min of 1 bar is used in the Controller.

In [ ]:
import pandapipes as pp
import numpy as np

def initialize_test_net(qext_w=np.array([100000, 200000]),
                        return_temperature=np.array([55, 60]),
                        supply_temperature=85, 
                        flow_pressure_pump=4,
                        lift_pressure_pump=1.5,
                        pipetype="110/202 PLUS"):
    
    print("Running the test network initialization script.")
    net = pp.create_empty_network(fluid="water")

    k = 0.1 # roughness defaults to 0.1

    suply_temperature_k = supply_temperature + 273.15
    return_temperature_k = return_temperature + 273.15

    # Junctions for pump
    j1 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 1", geodata=(0, 10))
    j2 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 2", geodata=(0, 0))

    # Junctions for connection pipes forward line
    j3 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 3", geodata=(10, 0))
    j4 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 4", geodata=(60, 0))

    # Junctions for heat exchangers
    j5 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 5", geodata=(85, 0))
    j6 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 6", geodata=(85, 10))
    
    # Junctions for connection pipes return line
    j7 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 7", geodata=(60, 10))
    j8 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 8", geodata=(10, 10))

    pump1 = pp.create_circ_pump_const_pressure(net, j1, j2, p_flow_bar=flow_pressure_pump, plift_bar=lift_pressure_pump, 
                                               t_flow_k=suply_temperature_k, type="auto", name="pump1")

    pipe1 = pp.create_pipe(net, j2, j3, std_type=pipetype, length_km=0.01, k_mm=k, name="pipe1", sections=5, text_k=283)
    pipe2 = pp.create_pipe(net, j3, j4, std_type=pipetype, length_km=0.05, k_mm=k, name="pipe2", sections=5, text_k=283)
    pipe3 = pp.create_pipe(net, j4, j5, std_type=pipetype, length_km=0.025,k_mm=k, name="pipe3", sections=5, text_k=283)

    heat_cosnumer1 = pp.create_heat_consumer(net, from_junction=j5, to_junction=j6, loss_coefficient=0, qext_w=qext_w[0], 
                                             treturn_k=return_temperature_k[0], name="heat_consumer_1") # treturn_k=t when implemented in function
    

    heat_cosnumer2 = pp.create_heat_consumer(net, from_junction=j4, to_junction=j7, loss_coefficient=0, qext_w=qext_w[1], 
                                             treturn_k=return_temperature_k[1], name="heat_consumer_2") # treturn_k=t when implemented in function
    
    pipe4 = pp.create_pipe(net, j6, j7, std_type=pipetype, length_km=0.25, k_mm=k, name="pipe4", sections=5, text_k=283)
    pipe5 = pp.create_pipe(net, j7, j8, std_type=pipetype, length_km=0.05, k_mm=k, name="pipe5", sections=5, text_k=283)
    pipe6 = pp.create_pipe(net, j8, j1, std_type=pipetype, length_km=0.01, k_mm=k, name="pipe6", sections=5, text_k=283)

    pp.pipeflow(net, mode="bidirectional", iter=100)

    return net

net = initialize_test_net()

dp_controller = BadPointPressureLiftController(net)
net.controller.loc[len(net.controller)] = [dp_controller, True, -1, -1, False, False]

pp.pipeflow(net, mode="bidirectional", iter=100)

You can now proceed to run time-series simulations or further analyses, and the controller will handle pressure adjustments as needed. 

Suggestions for improvements or alternative approaches are appreciated. Please feel free to contribute.